# 4 - Feras Formidáveis

## 4.9 A senha de tamanho variável

**Autores:** Os autores contribuíram igualmente, discutindo a resolução do exercício e os códigos

Enzo Januzzi Xavier: Revisão e texto explicativo

Guilherme Caetano de Almeida Sobreira: Maior elaboração do código

### Introdução:

O objetivo desta atividade é resolver o problema da senha em algoritmos genéticos sem forneçer a informação
do tamanho da senha para a função que gera a população, considerando que a senha pode ser uma string de 1 até 30 caracteres. Boa parte do código foi adaptada dos materiais de aula do Daniel Cassar [1,2], em que criou-se scripts específicos para o projeto, os quais serão comentados ao longo da resolução. O exercício foi realizado em conjunto com o Guilherme Sobreira, em que discutimos maneiras de modificar o algoritmo genético original.

### Desenvolvimento:

Importando as bibliotecas necessárias [3,4] e os scripts:

In [1]:
import random
from string import ascii_lowercase, ascii_uppercase, digits # importa as letras minúsculas, maiúsculas e dígitos
from Scripts import populacao_senha_size as cria_populacao
from Scripts import funcao_objetivo_pop_senha_size as funcao_objetivo
from Scripts import selecao_torneio_min as funcao_selecao
from Scripts import cruzamento_uniforme as funcao_cruzamento
from Scripts import mutacao_salto_tamanho as funcao_mutacao1
from Scripts import mutacao_simples as funcao_mutacao2
from Scripts import mutacao_salto as funcao_mutacao3

Definindo os hiperparâmetros do problema, dentre os quais criamos uma senha formada por uma lista com cada letra em string, além dos caracteres possíveis que formam uma senha, com letras maiúsculas, minúsculas e os dígitos

In [2]:
SENHA = list("Su1relyNotASecur6ePassword3")
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits

TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO_LETRA = 0.025
CHANCE_DE_MUTACAO_TAMANHO = 0.05
TAMANHO_TORNEIO = 3

Criando a população inicial do problema, em que cada indivíduo pode conter uma senha entre 1 e 30 caracteres:

In [3]:
populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)
#populacao

A função objetivo desse problema considera tanto a distância ordinal entre a letra do candidato e a letra da senha verdadeira, quanto a distância entre o tamanho do candidato e da senha. Aplicou-se um peso proporcional à diferença entre os extremos do ordinal para os caracteres possíveis, garantindo que o algoritmo encontre primeiro um canditado com o mesmo tamanho da senha verdadeira.

In [4]:
# Analisando o ordinal para os caracteres possíveis
print([letra for letra in CARACTERES_POSSIVEIS], '\n', [ord(letra) for letra in CARACTERES_POSSIVEIS])

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 
 [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]


In [5]:
# Definindo o peso
lista_ord = list(ord(letra) for letra in CARACTERES_POSSIVEIS)
peso = max(lista_ord) - min(lista_ord) + 1
peso

75

O código principal, contendo um laço while para fazer todas as etapas do algoritmo genético (seleção, cruzamento, mutação e atualização da geração) até encontrar a senha correta. Curiosamente, aplicar um cruzamento uniforme funcionou para encontrar a senha correta, após tentativas falhas de criar uma função de cruzamento específica para este problema. Além disso, aplicar 3 funções de mutação diferentes demonstrou ser benéfico pro treinamento do algoritmo, primeiro alterando o tamanho dos candidatos e depois seus genes específicos.

In [6]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:

    # Seleção
    fitness = funcao_objetivo(populacao, SENHA)
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)

    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)

    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO_TAMANHO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO_LETRA, list(CARACTERES_POSSIVEIS))
    funcao_mutacao3(proxima_geracao, CHANCE_DE_MUTACAO_LETRA, list(CARACTERES_POSSIVEIS))

    # Encerramento
    populacao = proxima_geracao
    geracao += 1

    fitness = funcao_objetivo(populacao, SENHA)
    menor_fitness_observado = min(fitness)

    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 eDlznJJ3ipC9xQm70ZO7cDJouF0
2 3nEqlaAIuGRFsThJEgNiJHglMZ
3 0nEaYalJuGRFX3hJVFgNlYevoAB
4 0rmznapJarC9xQh70FD7csevuSB
5 0rFznJpJirC9XQm70ZKQcsJouz
6 FnlqJalIapiBxQhL0eg8cssdoS0
7 FnPzYapJirCBXQhL0ZxQcYevoSB
8 0rEqnJlJrrFxQhmV0ZKQssdouj
9 FrOzlal3ivrVmXQhLrFgqbseolF
10 erEqnalJrrCFQQmV0eKNsssouj
11 erFznalJrrCFQhmV0eKNsssouj
14 FnFznapJrpCFxhmf0eKpsjsduj
15 enFqnapJrrCFQhmf0eDQsssouj
16 enFqnapJrrCFQhmf0eKQsssouj
17 FrFqnapJrpCTyhhf0eKpsssouj
18 FrFqnapJrpCTyhhu0eKpsssouj
19 FrFqnapJrpCTyhmu0eKpsssouj
20 FrFqnapJrrCTyhmu0eKpsssouj
22 FrFqnapJrrCTxhmu0eKpsssouj
24 FrFqnapJrpCTQhmu0eKksssouj
27 FrFqnapJrrCTQhmu0eKksssouj
28 GrFqndpJrrCTyhmu0eKksssouj
29 GrFqndqJrrCTyhmu0eKksssouj
30 FrFqnapJrrCTyhmu0eKaksssouj
31 Fr3qnapJrrCTQhmu0eKksssouj
32 Fr3qnapJrrCTyhmu0eKasssouj
33 Gr3qndpJrrCTyhmu0eKasssouj
34 Gr3rndqJrrCTyhmu0eKasssouj
39 Gr3rmdqJrrCTyhmu0eKasssouj
41 Gr3rneqJrrCTyhmu0eKasssoui
42 Gr3rmdqJqrCTyhmt0eKasssouj
45 Gr3rmeqJqrCTyhmu0eKasssoui
47 Gr3rmerJqrCTyhmu0eKasssoui
49 Gr3rmerJq

Perceba que mesmo com uma senha complexa, com 30 caracteres e misturando letras e números, o algoritmo conseguiu encontrá-la. Além disso, aplicar as 3 funções de mutação juntas demonstrou ser necessário tanto para a senha convergir quanto para diminuir o número de gerações necessárias. Fica a cargo do leitor verificar isso, alterando quais mutações serão aplicadas

### Conclusão:

Não fornecer a senha verdadeira para os indivíduos das gerações aumenta a dificuldade do problema, sendo necessário desenvolver outras estratégias para evoluir a população. Por outro lado, aprendi mais sobre esse tema, entendendo melhor como o algoritmo funciona e descobrindo que pode-se usar mais de uma função de mutação

### Referências:

[1] CASSAR, Daniel. "ATP-303 GA 4.2 - Notebook descobrindo a senha.ipynb". Material de Aula, 2025.

[2] CASSAR, Daniel. "funcoes.py". Scripts baseados, 2025

[3] Biblioteca Random. https://docs.python.org/3/library/random.html

[4] Biblioteca String. https://docs.python.org/3/library/string.html
